# **02 – Exploratory Data Analysis (EDA) Espacio-Temporal – ERA5 Precipitación (Chile)**

Este notebook realiza un análisis exploratorio avanzado de la base **ERA5**
(precipitación total diaria) para identificar patrones espacio-temporales,
comparar macrozonas y generar visualizaciones que servirán de insumo
para los modelos híbridos **AE-DMD / KoVAE**.

## **1. Cargar librerías y configuraciones**

In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from pathlib import Path

# Configuración estética
plt.style.use("seaborn-v0_8-whitegrid")
sns.set_palette("deep")
DATA_DIR = Path("../data/processed")
FIG_DIR = Path("../reports/figures")

## **2. Cargar datos procesados**

In [ ]:
# Dataset combinado (NetCDF)
nc_path = DATA_DIR / "era5_precipitation_chile_full.nc"
ds = xr.open_dataset(nc_path)
ds


## **3. Estadísticas básicas**

In [ ]:
print("Variables:", list(ds.data_vars))
print("Rango temporal:", str(ds.time.values[0]), "→", str(ds.time.values[-1]))
print("Dimensiones:", ds.dims)
print(ds["total_precipitation"])


## 4. **Mapa promedio espacial (Chile completo)**

In [ ]:
mean_precip = ds["total_precipitation"].mean(dim="time")

fig = plt.figure(figsize=(8, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
mean_precip.plot(ax=ax, cmap="Blues", cbar_kwargs={'label':'Precipitación media (m)'})
ax.add_feature(cfeature.BORDERS, linestyle='--', linewidth=0.5)
ax.add_feature(cfeature.COASTLINE)
ax.set_extent([-76, -66, -56, -17])
plt.title("Precipitación media 1980–2022 – ERA5 (Chile)")
plt.savefig(FIG_DIR / "era5_mean_map_chile.png", dpi=150, bbox_inches="tight")
plt.show()


## **5. Cargar series anuales por macrozona**

In [ ]:
zones = ["norte", "centro", "sur"]
dfs_annual = {z: pd.read_csv(DATA_DIR / f"era5_annual_{z}.csv") for z in zones}
dfs_monthly = {z: pd.read_csv(DATA_DIR / f"era5_monthly_{z}.csv") for z in zones}


## **6. Tendencias anuales por macrozona**

In [ ]:
plt.figure(figsize=(10,6))
for z in zones:
    df = dfs_annual[z]
    plt.plot(df["year"], df["total_precipitation"], marker="o", label=z.capitalize())
plt.title("Tendencias anuales de precipitación promedio (1980–2022)")
plt.xlabel("Año")
plt.ylabel("Precipitación promedio (m)")
plt.legend()
plt.savefig(FIG_DIR / "era5_trend_annual_zones.png", dpi=150, bbox_inches="tight")
plt.show()


## **7. Distribución mensual promedio (estacionalidad)**

In [ ]:
plt.figure(figsize=(9,5))
for z in zones:
    df = dfs_monthly[z]
    plt.plot(df["month"], df["total_precipitation"], marker="o", label=z.capitalize())
plt.title("Ciclo estacional de precipitación promedio – ERA5 (1980–2022)")
plt.xlabel("Mes")
plt.ylabel("Precipitación (m)")
plt.xticks(range(1,13))
plt.legend()
plt.savefig(FIG_DIR / "era5_monthly_cycle_zones.png", dpi=150, bbox_inches="tight")
plt.show()


## **8. Correlación entre macrozonas**

In [ ]:
merged = pd.DataFrame({
    "year": dfs_annual["centro"]["year"],
    "norte": dfs_annual["norte"]["total_precipitation"],
    "centro": dfs_annual["centro"]["total_precipitation"],
    "sur": dfs_annual["sur"]["total_precipitation"],
})
corr = merged.corr().round(2)
sns.heatmap(corr, annot=True, cmap="coolwarm", vmin=-1, vmax=1)
plt.title("Correlación de precipitaciones entre macrozonas")
plt.savefig(FIG_DIR / "era5_corr_zones.png", dpi=150, bbox_inches="tight")
plt.show()


## **9. Serie temporal global promedio**

In [ ]:
# Cargar CSV diario consolidado
df_daily = pd.read_csv(DATA_DIR / "era5_precipitation_chile_daily.csv")
df_daily["date"] = pd.to_datetime(df_daily["date"])
mean_daily = df_daily.groupby("date")["total_precipitation"].mean().rolling(30).mean()

plt.figure(figsize=(14,5))
plt.plot(mean_daily, color="royalblue")
plt.title("Serie temporal suavizada (media móvil 30 días) – Chile completo")
plt.xlabel("Año")
plt.ylabel("Precipitación media (m)")
plt.savefig(FIG_DIR / "era5_series_smooth.png", dpi=150, bbox_inches="tight")
plt.show()


## **10. Análisis exploratorio adicional**

In [ ]:
# Detección de años secos / húmedos
threshold = mean_daily.mean()
years_wet = mean_daily[mean_daily > threshold].index.year.unique()
years_dry = mean_daily[mean_daily < threshold].index.year.unique()

print("Años más húmedos:", years_wet[:10].tolist())
print("Años más secos:", years_dry[:10].tolist())


## **11. Exportar datos listos para modelado**

In [ ]:
# Exportar serie diaria promedio nacional
out_path = DATA_DIR / "era5_daily_national_mean.csv"
mean_daily.to_csv(out_path, header=["precipitation"])
print(f"✅ Archivo exportado: {out_path}")


### **Qué obtienes con este notebook**
- Mapas medios de precipitación (Chile completo).  
- Series temporales anuales y mensuales por macrozona.  
- Correlaciones entre regiones.  
- Visualización de tendencias y estacionalidad.  
- Dataset final preparado para alimentar tu pipeline de modelado.

---

### **Próximo paso**
Con este análisis listo, el siguiente bloque del proyecto es:

**Fase de modelado**  
- Cargar el dataset `era5_daily_national_mean.csv` (o las mallas interpoladas).  
- Entrenar el **Autoencoder (ae_keras.py)** para reducción de dimensionalidad.  
- Aplicar **DMD** o **KoVAE** sobre las representaciones latentes.  
- Integrar el entrenamiento con **MLflow** para comparar runs y métricas.